In [15]:
!pip install pandas
!pip install geopandas

     ---------------------------------------- 0.0/51.1 kB ? eta -:--:--
     ------------------------------ ------- 41.0/51.1 kB 960.0 kB/s eta 0:00:01
     -------------------------------------- 51.1/51.1 kB 648.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   -------- ------------------------------- 0.2/1.1 MB 6.9 MB/s eta 0:00:01
   ------------------- -------------------- 0.5/1.1 MB 5.6 MB/s eta 0:00:01
   -------------------------------- ------- 0.9/1.1 MB 7.1 MB/s eta 0:00:01
   ---------------------------------------  1.1/1.1 MB 7.8 MB/s eta 0:00:01
   ---------------------------------------- 1.1/1.1 MB 6.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/22.9 MB ? eta -:--:--
    --------------------------------------- 0.4/22.9 MB 8.5 MB/s eta 0:00:03
   - -------------------------------------- 0.8/22.9 MB 10.0 MB/s eta 0:00:03
   -- ------------------------------------- 1.2/22.9 MB 9.6 MB/s eta 0:00:03
   -- -----------

In [3]:
import pandas as pd
import requests
import geopandas as gpd
from os.path import isfile

### Téléchargement des données de consommation d'électricité annuelle par adresse

In [14]:
def get_data_consumption(url, year, replace:bool = False):
    '''Gets the data of home's electricity consumption
    Args:
        url(string)
        year(string)
        replace(bool): True to replace consommation{year}.csv if it exists. Defaults to False.
    Returns:
        df (dataframe): dataframe of the home's electricity consumption for the wanted year 
    '''
    path_to_data="consommation"+f"{year}"+".csv"
    if (isfile(path_to_data) and not replace):
        df=pd.read_csv(path_to_data, sep=";")
    else:
        print("Chargement des données, cette étape peut prendre quelques minutes")
        response=requests.get(url)
        if response.status_code == 200:
            with open(path_to_data, "wb") as file:
                file.write(response.content)
            print("Téléchargement réussi.")
        else:
            print(f"Échec du téléchargement. Code d'état : {response.status_code}")
        df=pd.read_csv(path_to_data, sep=";")
    return df


In [1]:
# URL des bases de données à charger 

consumption_data_url_2018="https://enedis.opendatasoft.com/api/explore/v2.1/catalog/datasets/consommation-annuelle-residentielle-par-adresse/exports/csv?lang=fr&refine=annee%3A%222018%22&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B"
consumption_data_url_2019= "https://enedis.opendatasoft.com/api/explore/v2.1/catalog/datasets/consommation-annuelle-residentielle-par-adresse/exports/csv?lang=fr&refine=annee%3A%222019%22&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B"
consumption_data_url_2020="https://enedis.opendatasoft.com/api/explore/v2.1/catalog/datasets/consommation-annuelle-residentielle-par-adresse/exports/csv?lang=fr&refine=annee%3A%222020%22&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B"
consumption_data_url_2021="https://enedis.opendatasoft.com/api/explore/v2.1/catalog/datasets/consommation-annuelle-residentielle-par-adresse/exports/csv?lang=fr&refine=annee%3A%222021%22&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B"

In [17]:
df_cons_2018=get_data_consumption(consumption_data_url_2018, "2018")
df_cons_2019=get_data_consumption(consumption_data_url_2019, "2019")
df_cons_2020=get_data_consumption(consumption_data_url_2020, "2020")
df_cons_2021=get_data_consumption(consumption_data_url_2021, "2021")

C:\Users\auran\AppData\Local\Temp\ipykernel_14392\3130095385.py:12: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(path_to_data, sep=";")


Chargement des données, cette étape peut prendre quelques minutes
Téléchargement réussi.


C:\Users\auran\AppData\Local\Temp\ipykernel_14392\3130095385.py:22: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(path_to_data, sep=";")


### Récupération des données de Diagnostics de Performance Energétique (DPE)

In [19]:
def get_dpe_from_url(api_url):
    '''Imports a database from an API url.
    Args:
        api_url (string) : url of the wanted API
    Returns:
        pandas.DataFrame : dataframe
    '''
    req=requests.get(api_url)
    wb=req.json()
    df = pd.json_normalize(wb["results"])
    dpe = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs = 4326)
    dpe = dpe.dropna(subset = ['longitude', 'latitude'])
    return dpe

In [21]:
# Il est trop couteux de récupérer l'intégralité des données et nous ne disposons de données météo que pour
# certaines communes. L'idée est donc de ne récupérer les données de DPE que pour les communes voulues et de
# les agréger dans une unique table de données.

def get_dpe_dataset(api_root, communes):
    